# LangGraph Lab: Building State-based LLM Workflows

This lab introduces LangGraph and guides students through building stateful graphs that orchestrate LLMs and tools. 

In [ ]:
%pip install langchain openai langchain-openai langchain-tavily langgraph langchain_experimental

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

OPENAI_MODEL = "gpt-5-mini"

## Mock LLM + simple StateGraph
This example builds a minimal `StateGraph` with a mock LLM node so you can see the graph structure and execution flow without calling a real API. 
- Understand nodes, edges, START/END and how messages flow through the graph.
- See how a node returns new messages that become the next state's input.

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph import add_messages
from langchain_core.messages import BaseMessage
from typing import TypedDict, Annotated, Sequence


def mock_llm(state: MessagesState):
    return {"messages": [{"role": "ai", "content": "hello world"}]}

graph = StateGraph(MessagesState)
graph.add_node(mock_llm)
graph.add_edge(START, "mock_llm")
graph.add_edge("mock_llm", END)
graph = graph.compile()

result = graph.invoke({"messages": [{"role": "user", "content": "hi!"}]})

for msg in result["messages"]:
    msg.pretty_print()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

## Integrating a real LLM node

Below is an example of how to wrap a real LLM call inside a node

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage


def openai_node(state: MessagesState):
    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [{"role": "ai", "content": response.content}]}

graph = StateGraph(MessagesState)
graph.add_node("llm_node", openai_node)
graph.add_edge(START, "llm_node")
graph.add_edge("llm_node", END)
graph = graph.compile()

print(type(graph)) # This should look familiar!

response = graph.invoke({"messages": [{"role": "user", "content": "Why is Amir the best TA?"}]})
for msg in response["messages"]:
    msg.pretty_print()

## Combining LLMs with tool nodes and routing
This section shows how to bind tools to an LLM and add a `ToolNode` into the graph with conditional routing. 

Notice that this workflow is **very similar to an agent in LangChain**, where a model decides the next step based on input. However, with **LangGraph** we gain **much more fine-grained control** over the flow and state of execution:

- **Explicit state management**: Every node receives and returns a well-defined `State` object, making it easier to track, debug, and persist intermediate results.  
- **Conditional routing**: You can define precise routing logic between nodes, rather than relying solely on model outputs to decide the next action.  
- **Composability**: Nodes are simple, independent functions that can be reused across multiple graphs, allowing you to mix and match logic without rewriting agents.  
- **Observability**: The graph structure makes it easier to visualize and understand complex workflows, which is harder to achieve in traditional agent chains.  
- **Extensibility**: Adding new behaviors, categories, or processing steps is as simple as adding a new node and connecting it with edges or conditional logic.

This **bridges the gap between agent-like reasoning and structured workflow orchestration**, enabling applications that are both **intelligent** and **robust**.


In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_tavily import TavilySearch

os.environ["TAVILY_API_KEY"] = "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"

tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    include_answer=True
)

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

tools = [tavily_search_tool, get_weather]

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)
llm_with_tools = llm.bind_tools(tools)

def openai_node(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

tool_node = ToolNode(tools)

# Define the function that determines whether to continue or not
def router(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there are no tool calls, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "tool"

graph = StateGraph(MessagesState)
graph.add_node("llm_node", openai_node)
graph.add_node("tool_node", tool_node)

# Set the entrypoint as `llm_node`
# This means that this node is the first one called
graph.set_entry_point("llm_node")

# We now add a conditional edge
graph.add_conditional_edges(
    # First, we define the start node. We use `llm_node`.
    # This means these are the edges taken after the `llm_node` is called.
    "llm_node",
    # Next, we pass in the function that will determine which node is called next.
    router,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `router`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "tool": "tool_node",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `llm_node`.
# This means that after `tools` is called, `llm_node` is called next.
graph.add_edge("tool_node", "llm_node")

graph = graph.compile()

response = graph.invoke({"messages": [{"role": "user", "content": "What's the weather in New York City"}]})
for msg in response["messages"]:
    msg.pretty_print()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

## Graph-based routing example — intent and behavior

This example demonstrates a small workflow that classifies text as either a **"compliment"** or a **"question"** and routes execution accordingly.
This portion is inspired by this Medium article: https://levelup.gitconnected.com/gentle-introduction-to-langgraph-a-step-by-step-tutorial-2b314c967d3c

### Why this pattern is powerful
This approach showcases how **graph-based workflows** can dynamically adapt based on intent or content, a concept that scales far beyond simple classification.  
You can apply this same routing logic to:
- **Customer feedback systems** — route praise to marketing, questions to support, and complaints to escalation teams.  
- **Multi-agent systems** — direct tasks to the most capable agent based on detected intent.  
- **Automated pipelines** — trigger different tools or APIs depending on user input or detected context.  

By modularizing logic into nodes and connecting them conditionally, you create **scalable, interpretable, and testable pipelines** that are easy to extend with new categories or behaviors.


In [ ]:
from langgraph.graph import StateGraph
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import cast
from langgraph.graph import END, START

class State(TypedDict):
    text: str
    answer: str
    payload: dict[str, list]
    tag: str

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)    
template = """
I have a piece of text: {text}. 
Tell me whether it is a 'compliment' or a 'question'. 
"""
prompt = ChatPromptTemplate([("user", template)])
chain = prompt | llm | StrOutputParser()

# here we will use an llm to route our feedback into categories of compliment or question
# right now we are assuming anything that is not a compliment is a question
def route_question_or_compliment(state: State):
    response = chain.invoke({"text": state["text"]})
    return "compliment" if "compliment" in response.lower() else "question"

# in reality this may be sent to a database or external system
def run_compliment_code(state: State):  
    return {"answer": "Thanks for the compliment."}

# in reality this may be sent to a customer support system
def run_question_code(state: State):
    return {"answer": "Wow nice question."}

# this is just an arbitrary indication that the feedback has been processed
def mark_as_completed(state: State):
    return {"answer": [state["answer"] + " Acknowledged."]}

# while routing questions or compliments, we may also want to tag them based on their content
def tag_query(state: State):
    if "package" in state["text"]:
        return {"tag": "Packaging"}
    elif "price" in state["text"]:
        return {"tag": "Pricing"}
    else:
        return {"tag": "General"}

def extract_content(state: State):
    return {"text": state["payload"].get("customer_remark", "")}

graph_builder = StateGraph(State)
graph_builder.add_node("extract_content", extract_content)
graph_builder.add_node("run_question_code", run_question_code)
graph_builder.add_node("run_compliment_code", run_compliment_code)
graph_builder.add_node("mark_as_completed", mark_as_completed)

graph_builder.add_edge(START, "extract_content")
graph_builder.add_edge("run_question_code", "mark_as_completed")
graph_builder.add_edge("run_compliment_code", "mark_as_completed")
graph_builder.add_edge("mark_as_completed", END)

graph_builder.add_node("tag_query", tag_query)
graph_builder.add_edge("tag_query", "mark_as_completed")

graph_builder.add_conditional_edges(
    "extract_content",
    route_question_or_compliment,
    {
        "compliment": "run_compliment_code",
        "question": "run_question_code",
    },
)
graph_builder.add_edge("extract_content", "tag_query")
graph = graph_builder.compile()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))


## Running the graph and inspecting results
- Observe how state is transformed as the graph runs.
- Inspect `result_state` to verify nodes produced expected outputs.
- Try different input payloads to exercise different conditional branches.

In [ ]:
result_state = graph.invoke(cast(State, {
    "text": "",
    "answer": "",
    "payload": {
        "time_of_comment": "20-01-2025",
        "customer_remark": "I love your product and the price is great!",
        "social_media_channel": "facebook",
        "number_of_likes": 100,
    },
    "tag": "",
}))

print(result_state)

## Advanced: Python REPL tool and multi-agent collaboration
This larger cell defines helper tools and two agents (researcher and chart generator). Learning goals:
- See how you can wrap complex behaviors (search, code execution) into tools that agents call.
- Observe how agents can hand off work to other agents and coordinate via message histories.


### Importance of Multiple Agents

Using multiple specialized agents adheres to the **Single Responsibility Principle (SRP)**, each agent focuses on one specific task (e.g., research, analysis, visualization).  
This separation improves **modularity**, making each component easier to maintain, test, and extend.

It also enhances **performance and scalability**: agents can operate in parallel or sequentially, each leveraging tools and models best suited for their role.  
For instance, a **research agent** may gather data from the web while a **chart generator** agent handles visualization, together enabling faster, more reliable, and more adaptable workflows.
Additionally I have personally found that multiple agents that each perform a specific task often performs better than asking a single agent to do multiple tasks, especially with smaller LLMS



In [ ]:
from typing import Annotated, Literal

from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI
import io
from contextlib import redirect_stdout
# from langchain_experimental.utilities.python import PythonREPL
class PythonREPL:
    def run(self, code: str):
        """Execute code and return captured stdout as a string.
        Errors are propagated to the caller (so callers can format error messages)."""
        buf = io.StringIO()
        globals_dict = {"__name__": "__main__"}
        try:
            with redirect_stdout(buf):
                exec(code, globals_dict)
        except BaseException:
            raise
        return buf.getvalue()

from langchain_core.messages import BaseMessage, HumanMessage
from langchain.agents import create_agent
from langgraph.graph import MessagesState, END
from langgraph.types import Command


tavily_tool = TavilySearch(max_results=2)
# I am hard coding this because tavily has been giving me some issues
def search_tool(query: str) -> str:
    """Use this to search for answers to a given query"""
    return (
        "The ethnic composition of the United States (based on 2020 Census data) is approximately:\n"
        "- White (non-Hispanic): 57.8%\n"
        "- Hispanic or Latino: 18.7%\n"
        "- Black or African American: 12.1%\n"
        "- Asian: 5.9%\n"
        "- Native American and Alaska Native: 0.7%\n"
        "- Native Hawaiian and Other Pacific Islander: 0.2%\n"
        "- Two or more races: 4.6%\n\n"
    )

repl = PythonREPL()

def python_repl_tool(
    code: Annotated[str, "The Python code to execute to generate your chart."],
):
    """Execute Python code and display matplotlib charts inline when possible.
    Falls back to saving the figure if the environment is non-interactive.
    """
    import matplotlib
    import io
    import sys

    buf = io.StringIO()
    globals_dict = {"__name__": "__main__"}

    # Try to detect if we can show plots
    interactive_backends = {"TkAgg", "Qt5Agg", "MacOSX", "inline"}
    current_backend = matplotlib.get_backend()

    # If the backend is non-interactive, switch to inline display (for notebooks)
    if current_backend not in interactive_backends:
        matplotlib.use("Agg")

    try:
        # Capture stdout from the executed code
        with redirect_stdout(buf):
            exec(code, globals_dict)

        # Try showing the figure if interactive
        import matplotlib.pyplot as plt
        if current_backend in interactive_backends:
            plt.show()
        else:
            plt.savefig("output_chart.png", dpi=300)
            print("Non-interactive environment detected — saved chart as png")

    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"

    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {buf.getvalue()}"
    return (
        result_str
        + "\n\n If you have completed all tasks, respond with FINAL ANSWER."
    )


def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        f"\n{suffix}"
    )

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)


def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto

research_agent = create_agent(
    llm,
    tools=[search_tool],
    system_prompt=make_system_prompt(
        "You are responsible for finding accurate, concise data for the request. Return usable results that your charting colleague can directly visualize."
    ),
)


def research_node(
    state: MessagesState,
) -> Command[Literal["chart_generator", END]]:
    result = research_agent.invoke(state)
    goto = "chart_generator"
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="researcher"
    )
    return Command(
        update={
            # share internal message history of research agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )


chart_agent = create_agent(
    llm,
    [python_repl_tool],
    system_prompt=make_system_prompt(
        "You are responsible for creating charts or visual summaries using provided data. Execute the Python code directly to generate the chart. After successfully producing the chart, respond with 'FINAL ANSWER' and stop."
    ),
)


def chart_node(state: MessagesState) -> Command[Literal["researcher", END]]:
    result = chart_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "researcher")

    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="chart_generator"
    )
    return Command(
        update={
            # share internal message history of chart agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(MessagesState)
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

workflow.add_edge(START, "researcher")
graph = workflow.compile()

from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
events = graph.stream(
    {
        "messages": [
            (
                "user",
                "Find data on the ethnic diversity of the U.S. population and create a pie chart showing the percentage of each group."+
                "Once you make the chart, finish.",
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 15},
)

for msg in events:
    key = next((k for k in ["researcher", "chart_generator"] if k in msg), None)
    
    if key:
        messages = msg[key].get("messages", [])
        print(f"--- {key} ---")
        if messages:
            print(messages[-1].content)

#ONCE THIS HAS COMPLETED, CHECK YOUR FILE SYSTEM IF THE CHART DOES NOT SHOW BELOW


## Challange: Make your own langgraph system

Try to think of a problem that could be solved with an AI agent and mock out how it could be done. Think of something like the example above where we handled customer feedback

In [ ]:
# TODO: Create your own langgraph system